In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [3]:
def log_odds_to_prob(l):
    return 1 - (1 / (1 + np.exp(l)))

def prob_to_log_odds(p):
    return np.log(p / (1 - p))

In [4]:
laser_content = sio.loadmat('./data/laser_struct.mat')
laser = laser_content['laser']

In [5]:
poses = laser['pose'][0]
poses = np.matrix([arr[0].tolist() for arr in poses])

In [6]:
# Some setup
prior = 0.50
prob_occ = 0.9
prob_free = 0.35

grid_size = 0.5

border = 30
pose_x_min = np.min(poses[:, 0])
pose_x_max = np.max(poses[:, 0])
pose_y_min = np.min(poses[:, 1])
pose_y_max = np.max(poses[:, 1])

map_borders = (pose_x_min-border, pose_x_max+border,
               pose_y_min-border, pose_y_max+border)

offset_x = map_borders[0]
offset_y = map_borders[2]
offset = (offset_x, offset_y)

map_size_meters = (map_borders[1]-offset_x, map_borders[3]-offset_y)
map_size = tuple([math.ceil(dim/grid_size) for dim in map_size_meters])

log_odds_prior = prob_to_log_odds(prior)
grid_map = np.ones(map_size) * log_odds_prior

In [7]:
def world_to_map_coordinates(world_points, grid_size, offset):
    ofx, ofy = offset
    map_points = np.zeros(world_points.shape)
    
    for i in range(map_points.shape[1]):
        col = world_points[:, [i]]
        map_points[:, [i]] = np.array([
            [math.ceil((col[0] - ofx) / grid_size)],
            [math.ceil((col[1] - ofy) / grid_size)]
        ])
        
    return map_points

In [8]:
pose = poses[16].T

print(world_to_map_coordinates(pose[:2, :], grid_size, offset))

[[  67.]
 [ 124.]]
